In [1]:
import random,math,matplotlib.pyplot as plt,seaborn as sns,numpy as np,pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn import linear_model
from sklearn.model_selection import train_test_split

In [2]:
def readfile():#function to readfile
       df=pd.read_csv('Bengaluru_House_Data.csv')
       return df

df=readfile()
print(df.head().to_string())

              area_type   availability                  location       size  society total_sqft  bath  balcony   price
0  Super built-up  Area         19-Dec  Electronic City Phase II      2 BHK  Coomee        1056   2.0      1.0   39.07
1            Plot  Area  Ready To Move          Chikka Tirupathi  4 Bedroom  Theanmp       2600   5.0      3.0  120.00
2        Built-up  Area  Ready To Move               Uttarahalli      3 BHK      NaN       1440   2.0      3.0   62.00
3  Super built-up  Area  Ready To Move        Lingadheeranahalli      3 BHK  Soiewre       1521   3.0      1.0   95.00
4  Super built-up  Area  Ready To Move                  Kothanur      2 BHK      NaN       1200   2.0      1.0   51.00


In [3]:
df.shape

(13320, 9)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     13320 non-null  object 
 1   availability  13320 non-null  object 
 2   location      13319 non-null  object 
 3   size          13304 non-null  object 
 4   society       7818 non-null   object 
 5   total_sqft    13320 non-null  object 
 6   bath          13247 non-null  float64
 7   balcony       12711 non-null  float64
 8   price         13320 non-null  float64
dtypes: float64(3), object(6)
memory usage: 936.7+ KB


In [5]:
df.isnull().sum()

area_type          0
availability       0
location           1
size              16
society         5502
total_sqft         0
bath              73
balcony          609
price              0
dtype: int64

In [6]:
#As we dont need area type,balcony,society we drop them
df.drop(['area_type','society','balcony'],axis=1,inplace=True)

In [7]:
df.describe()

,bath,price
count,13247.000000,13320.000000
mean,2.692610,112.565627
std,1.341458,148.971674
min,1.000000,8.000000
25%,2.000000,50.000000
50%,2.000000,72.000000
75%,3.000000,120.000000
max,40.000000,3600.000000


In [8]:
df.isnull().sum()

availability     0
location         1
size            16
total_sqft       0
bath            73
price            0
dtype: int64

In [9]:
#to make location compeletly fill
print(df['location'].value_counts().to_string())

location
Whitefield                                            540
Sarjapur  Road                                        399
Electronic City                                       302
Kanakpura Road                                        273
Thanisandra                                           234
Yelahanka                                             213
Uttarahalli                                           186
Hebbal                                                177
Marathahalli                                          175
Raja Rajeshwari Nagar                                 171
Bannerghatta Road                                     152
Hennur Road                                           152
7th Phase JP Nagar                                    149
Haralur Road                                          142
Electronic City Phase II                              132
Rajaji Nagar                                          107
Chandapura                                            100
Bella

In [10]:
#as most is for white field we put the null value as whitefield
df['location']=df['location'].fillna('Whitefield')
df.isnull().sum()

availability     0
location         0
size            16
total_sqft       0
bath            73
price            0
dtype: int64

In [11]:
#now for size
print(df['size'].value_counts().to_string())

size
2 BHK         5199
3 BHK         4310
4 Bedroom      826
4 BHK          591
3 Bedroom      547
1 BHK          538
2 Bedroom      329
5 Bedroom      297
6 Bedroom      191
1 Bedroom      105
8 Bedroom       84
7 Bedroom       83
5 BHK           59
9 Bedroom       46
6 BHK           30
7 BHK           17
1 RK            13
10 Bedroom      12
9 BHK            8
8 BHK            5
11 BHK           2
11 Bedroom       2
10 BHK           2
14 BHK           1
13 BHK           1
12 Bedroom       1
27 BHK           1
43 Bedroom       1
16 BHK           1
19 BHK           1
18 Bedroom       1


In [12]:
df['size']=df['size'].fillna('2 BHK')

In [13]:
df['bath']=df['bath'].fillna(df['bath'].median())

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   availability  13320 non-null  object 
 1   location      13320 non-null  object 
 2   size          13320 non-null  object 
 3   total_sqft    13320 non-null  object 
 4   bath          13320 non-null  float64
 5   price         13320 non-null  float64
dtypes: float64(2), object(4)
memory usage: 624.5+ KB


In [15]:
df['bhk']=df['size'].str.split().str.get(0).astype(int)

In [16]:
df[df['bhk']>20]

,availability,location,size,total_sqft,bath,price,bhk
1718,Ready To Move,2Electronic City Phase II,27 BHK,8000,27.0,230.0,27
4684,Ready To Move,Munnekollal,43 Bedroom,2400,40.0,660.0,43


In [17]:
print(df['total_sqft'].unique())

['1056' '2600' '1440' ... '1133 - 1384' '774' '4689']


In [18]:
def convert(x):
    temp=x.split('-')
    if(len(temp)==2):
        return (float(temp[0])+float(temp[1]))/2
    try:
        return float(x)
    except:
        return None

In [19]:
df['total_sqft']=df['total_sqft'].apply(convert)

In [20]:
df.head()

,availability,location,size,total_sqft,bath,price,bhk
0,19-Dec,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2
1,Ready To Move,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4
2,Ready To Move,Uttarahalli,3 BHK,1440.0,2.0,62.00,3
3,Ready To Move,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3
4,Ready To Move,Kothanur,2 BHK,1200.0,2.0,51.00,2


PRICE PER SQUARE FEET

In [21]:
df['price_sqft']=df['price']*100000 /df['total_sqft']

In [22]:
df['price_sqft']

0         3699.810606
1         4615.384615
2         4305.555556
3         6245.890861
4         4250.000000
             ...     
13315     6689.834926
13316    11111.111111
13317     5258.545136
13318    10407.336319
13319     3090.909091
Name: price_sqft, Length: 13320, dtype: float64

In [23]:
df.describe()

,total_sqft,bath,price,bhk,price_sqft
count,13274.000000,13320.000000,13320.000000,13320.000000,1.327400e+04
mean,1559.626694,2.688814,112.565627,2.802778,7.907501e+03
std,1238.405258,1.338754,148.971674,1.294496,1.064296e+05
min,1.000000,1.000000,8.000000,1.000000,2.678298e+02
25%,1100.000000,2.000000,50.000000,2.000000,4.266865e+03
50%,1276.000000,2.000000,72.000000,3.000000,5.434306e+03
75%,1680.000000,3.000000,120.000000,3.000000,7.311746e+03
max,52272.000000,40.000000,3600.000000,43.000000,1.200000e+07


In [24]:
df['location']=df['location'].apply(lambda x:x.strip())
location_count=df['location'].value_counts()

In [25]:
location_count_less_10=location_count[location_count<=10]
location_count_less_10

location
Dairy Circle                      10
Nagappa Reddy Layout              10
Basapura                          10
1st Block Koramangala             10
Sector 1 HSR Layout               10
                                  ..
Bapuji Layout                      1
1st Stage Radha Krishna Layout     1
BEML Layout 5th stage              1
singapura paradise                 1
Abshot Layout                      1
Name: count, Length: 1053, dtype: int64

In [26]:
df['location']=df['location'].apply(lambda x: 'other' if x in location_count_less_10 else x)


OUTLIER REMOVAL

In [27]:
df.describe()

,total_sqft,bath,price,bhk,price_sqft
count,13274.000000,13320.000000,13320.000000,13320.000000,1.327400e+04
mean,1559.626694,2.688814,112.565627,2.802778,7.907501e+03
std,1238.405258,1.338754,148.971674,1.294496,1.064296e+05
min,1.000000,1.000000,8.000000,1.000000,2.678298e+02
25%,1100.000000,2.000000,50.000000,2.000000,4.266865e+03
50%,1276.000000,2.000000,72.000000,3.000000,5.434306e+03
75%,1680.000000,3.000000,120.000000,3.000000,7.311746e+03
max,52272.000000,40.000000,3600.000000,43.000000,1.200000e+07


In [28]:
(df['total_sqft']/df['bhk']).describe()#to see only two column describe

count    13274.000000
mean       575.074878
std        388.205175
min          0.250000
25%        473.333333
50%        552.500000
75%        625.000000
max      26136.000000
dtype: float64

In [29]:
data=df[(df['total_sqft']/df['bhk']) >= 300]#to make another data with the given qualities
data.describe()

,total_sqft,bath,price,bhk,price_sqft
count,12530.000000,12530.000000,12530.000000,12530.000000,12530.000000
mean,1594.564544,2.559537,111.382401,2.650838,6303.979357
std,1261.271296,1.077938,152.077329,0.976678,4162.237981
min,300.000000,1.000000,8.440000,1.000000,267.829813
25%,1116.000000,2.000000,49.000000,2.000000,4210.526316
50%,1300.000000,2.000000,70.000000,3.000000,5294.117647
75%,1700.000000,3.000000,115.000000,3.000000,6916.666667
max,52272.000000,16.000000,3600.000000,16.000000,176470.588235


In [30]:
data.shape

(12530, 8)

In [31]:
data.price_sqft.describe()

count     12530.000000
mean       6303.979357
std        4162.237981
min         267.829813
25%        4210.526316
50%        5294.117647
75%        6916.666667
max      176470.588235
Name: price_sqft, dtype: float64

In [32]:
def remoutlier_sqft(df):
    df_res=pd.DataFrame()
    for a,b in df.groupby('location'):
        m=np.mean(b.price_sqft)
        st=np.std(b.price_sqft)
        new_df=b[(b.price_sqft > (m-st)) & (b.price_sqft <= m+st)]
        df_res=pd.concat([df_res,new_df],ignore_index=True)
    return df_res

data=remoutlier_sqft(data)
data.describe()

,total_sqft,bath,price,bhk,price_sqft
count,10301.000000,10301.000000,10301.000000,10301.000000,10301.000000
mean,1508.440608,2.471702,91.286372,2.574896,5659.062876
std,880.694214,0.979449,86.342786,0.897649,2265.774749
min,300.000000,1.000000,10.000000,1.000000,1250.000000
25%,1110.000000,2.000000,49.000000,2.000000,4244.897959
50%,1286.000000,2.000000,67.000000,2.000000,5175.600739
75%,1650.000000,3.000000,100.000000,3.000000,6428.571429
max,30400.000000,16.000000,2200.000000,16.000000,24509.803922


In [33]:
def remove_outlier_bhk(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('bhk'):
            bhk_stats [bhk] = {
                'mean': np.mean(bhk_df.price_sqft),
                'std': np.std(bhk_df.price_sqft),
                'count': bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('bhk'):
            stats =bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')

In [34]:
data=remove_outlier_bhk(data)

In [35]:
data.shape

(7361, 8)

In [36]:
data.drop('availability',axis=1,inplace=True)

In [37]:
data.shape

(7361, 7)

In [38]:
data

,location,size,total_sqft,bath,price,bhk,price_sqft
0,1st Block Jayanagar,4 BHK,2850.0,4.0,428.0,4,15017.543860
1,1st Block Jayanagar,3 BHK,1630.0,3.0,194.0,3,11901.840491
2,1st Block Jayanagar,3 BHK,1875.0,2.0,235.0,3,12533.333333
3,1st Block Jayanagar,3 BHK,1200.0,2.0,130.0,3,10833.333333
4,1st Block Jayanagar,2 BHK,1235.0,2.0,148.0,2,11983.805668
...,...,...,...,...,...,...,...
10292,other,2 BHK,1200.0,2.0,70.0,2,5833.333333
10293,other,1 BHK,1800.0,1.0,200.0,1,11111.111111
10296,other,2 BHK,1353.0,2.0,110.0,2,8130.081301
10297,other,1 Bedroom,812.0,1.0,26.0,1,3201.970443


CLEANED DATA

In [39]:
data.head()

,location,size,total_sqft,bath,price,bhk,price_sqft
0,1st Block Jayanagar,4 BHK,2850.0,4.0,428.0,4,15017.543860
1,1st Block Jayanagar,3 BHK,1630.0,3.0,194.0,3,11901.840491
2,1st Block Jayanagar,3 BHK,1875.0,2.0,235.0,3,12533.333333
3,1st Block Jayanagar,3 BHK,1200.0,2.0,130.0,3,10833.333333
4,1st Block Jayanagar,2 BHK,1235.0,2.0,148.0,2,11983.805668


In [40]:
data.to_csv('Cleaned_data.csv')

In [41]:
x=data.drop(['price','size'],axis=1)
y=data['price']

In [54]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.metrics import r2_score
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [55]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [56]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(5888, 5)
(1473, 5)
(5888,)
(1473,)


LINEAR REGRESSION

In [57]:
x_train.location=le.fit_transform(x_train.location)

In [58]:
scalar=StandardScaler()

In [59]:
lr=LinearRegression()

In [60]:
lr.fit(x_train,y_train)

LinearRegression()

In [61]:
linearaccuracy=lr.score(x_train,y_train)
linearaccuracy

0.9240754242418943

In [62]:
print("ACCCURACY = ",round(linearaccuracy,5)*100,"%")

ACCCURACY =  92.408 %


LASSO REGRESSION

In [63]:
from sklearn.linear_model import Lasso
lasso=Lasso()

In [64]:
lasso.fit(x_train,y_train)

Lasso()

In [65]:
lasso_accuracy=lasso.score(x_train,y_train)
lasso_accuracy

0.9237156662450755

In [66]:
print("ACCCURACY = ",round(lasso_accuracy,5)*100,"%")

ACCCURACY =  92.372 %


RIDGE REGRESSION

In [67]:
from sklearn.linear_model import Ridge

In [68]:
ridge=Ridge()

In [69]:
ridge.fit(x_train,y_train)

Ridge()

In [70]:
ridge_accuracy=ridge.score(x_train,y_train)
ridge_accuracy

0.9240754224268757

In [71]:
print("ACCCURACY = ",round(ridge_accuracy,5)*100,"%")

ACCCURACY =  92.408 %


CHECKING BEST ACCURACY

In [72]:
print("LINEAR REGRESSION ACCCURACY = ",round(linearaccuracy,5)*100,"%")#lr
print("LASSO ACCCURACY = ",round(lasso_accuracy,5)*100,"%")#lasso
print("RIDGE ACCCURACY = ",round(ridge_accuracy,5)*100,"%")#ridge

LINEAR REGRESSION ACCCURACY =  92.408 %
LASSO ACCCURACY =  92.372 %
RIDGE ACCCURACY =  92.408 %


In [73]:
import pickle
pickle.dump(lr,open('LR_model.pkl','wb'))

DECISION TREE

In [74]:
from sklearn import tree
tr=tree.DecisionTreeClassifier()

In [75]:
x.columns

Index(['location', 'total_sqft', 'bath', 'bhk', 'price_sqft'], dtype='object')

In [76]:
x["total_sqft_new"]=le.fit_transform(x['total_sqft'])

In [77]:
x["bath_new"]=le.fit_transform(x['bath'])
x["bhk_new"]=le.fit_transform(x['bhk'])
x["price_sqft_new"]=le.fit_transform(x['price_sqft'])
x["location_new"]=le.fit_transform(x['location'])

In [78]:
xdecision=x.drop(['total_sqft','bath','bhk','price_sqft','location'],axis='columns')

In [79]:
xdecision

,total_sqft_new,bath_new,bhk_new,price_sqft_new,location_new
0,1338,3,3,4700,0
1,858,2,2,4570,0
2,1038,1,2,4611,0
3,481,1,2,4495,0
4,515,1,1,4575,0
...,...,...,...,...,...
10292,481,1,1,2497,241
10293,990,0,0,4515,241
10296,636,1,1,4117,241
10297,169,0,0,157,241


In [80]:
tr.fit(xdecision,y)

ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.